# LTBP Utils Functions

> Functions Used In Modeling Efforts

In [ ]:
#| default_exp data.utils

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

from data_system_utilities.snowflake.query import Snowflake
from data_system_utilities.snowflake.copyinto import sf_to_adls_url_query_generator
from data_system_utilities.file_parsers import yaml
from machine_learning_utilities.dataset_creation.snowflake import select_static_features, select_additional_features, create_base_query

from fastcore.xtras import is_listy

import LTBP.files as files
import os
import logging
import sys

In [ ]:
#| export


def get_yaml_dicts(
    yaml_file_names: list  # list of yaml file(s) names that are all in the yaml_files section of the repo
):
    """
    Give a list of files to this function from the /files
    in a library and it will read/Parse yaml files and
    return a list of dictionaries to unpack
    """
    yaml_file_names = yaml_file_names if is_listy(yaml_file_names) else list(yaml_file_names)
    yaml_dicts = []
    for yf in yaml_file_names:
        yaml_dict = yaml.yaml_reader(os.path.join(files.__path__[0], 'yaml_files', yf))
        yaml_dicts.append(yaml_dict)
    return yaml_dicts

In [ ]:
features, inputs = get_yaml_dicts(['features.yaml', 'udf_inputs.yaml'])

In [ ]:
#| hide
test_eq(inputs.keys(), ['INFERENCE', 'TRAINING'])

In [ ]:
#| export


def generate_data_lake_query(
    stage_name: str,  # name of the sf stage to be created is just a file location pointer
    stage_path: str,  # adls file path location to start the stage location
    columns: list,  # columns to query parquet files in adls
    header=True,  # parquet files have headers
    extra_statement: str = None  # adding sql statements if desired
):
    """
    Given the columns names are provided this query will query out parquet data
    from azure datalake all in varchar this is the basic approach.
    """
    query = f"""
    select
    FEATURES_HERE
    from @{os.path.join(stage_name, stage_path)}
    {extra_statement}
    """
    if header is True:
        for ind, feature in enumerate(columns):
            if ind == 0:
                query = query.replace(
                    "FEATURES_HERE",
                    f'$1:"{feature}"::varchar as {feature}\nFEATURES_HERE',
                )
            else:
                query = query.replace(
                    "FEATURES_HERE",
                    f', $1:"{feature}"::varchar as {feature}\nFEATURES_HERE',
                )
    else:
        for ind, feature in enumerate(columns):
            if ind == 0:
                query = query.replace(
                    "FEATURES_HERE",
                    f'$1:"_COL_{ind}"::varchar as {feature}\nFEATURES_HERE',
                )
            else:
                query = query.replace(
                    "FEATURES_HERE",
                    f', $1:"_COL_{ind}"::varchar as {feature}\nFEATURES_HERE',
                )
    query = query.replace("FEATURES_HERE", "")
    return query

In [ ]:
columns = ['destinationgeoafinitylabel','evercorepass','everpass','gendercode','guestbehavior',
           'isepicmixactivated','marketingzone','mostcommonticketcomp','mostsubseasonvisited',
           'mostvisitedregion','mostvisitedresort','onlysingleresortkey','partnerresortscannerflag',
           'skierabilitylabel','totalseasonsscanned','visitmostinpeak','age','avgvisitperseason',
           'resortsvisited','subseasonsperyear','totalvisits']

query = generate_data_lake_query(stage_name='LTBPLocalRunTest',
     stage_path='projects/LTBP/FY23/experiments/<EXPERIMENT_NAME>/training_data/'.replace('<EXPERIMENT_NAME>', 'ltbp_nbs_testing'),
     columns=columns,
     extra_statement=None)
print(query)


    select
    $1:"destinationgeoafinitylabel"::varchar as destinationgeoafinitylabel
, $1:"evercorepass"::varchar as evercorepass
, $1:"everpass"::varchar as everpass
, $1:"gendercode"::varchar as gendercode
, $1:"guestbehavior"::varchar as guestbehavior
, $1:"isepicmixactivated"::varchar as isepicmixactivated
, $1:"marketingzone"::varchar as marketingzone
, $1:"mostcommonticketcomp"::varchar as mostcommonticketcomp
, $1:"mostsubseasonvisited"::varchar as mostsubseasonvisited
, $1:"mostvisitedregion"::varchar as mostvisitedregion
, $1:"mostvisitedresort"::varchar as mostvisitedresort
, $1:"onlysingleresortkey"::varchar as onlysingleresortkey
, $1:"partnerresortscannerflag"::varchar as partnerresortscannerflag
, $1:"skierabilitylabel"::varchar as skierabilitylabel
, $1:"totalseasonsscanned"::varchar as totalseasonsscanned
, $1:"visitmostinpeak"::varchar as visitmostinpeak
, $1:"age"::varchar as age
, $1:"avgvisitperseason"::varchar as avgvisitperseason
, $1:"resortsvisited"::varchar a

In [ ]:
#| hide
test_eq(query.replace(' ', '').replace('\n', ''), """select    $1:"destinationgeoafinitylabel"::varchar as destinationgeoafinitylabel, $1:"evercorepass"::varchar as evercorepass, $1:"everpass"::varchar as everpass, $1:"gendercode"::varchar as gendercode, $1:"guestbehavior"::varchar as guestbehavior, $1:"isepicmixactivated"::varchar as isepicmixactivated, $1:"marketingzone"::varchar as marketingzone, $1:"mostcommonticketcomp"::varchar as mostcommonticketcomp, $1:"mostsubseasonvisited"::varchar as mostsubseasonvisited, $1:"mostvisitedregion"::varchar as mostvisitedregion, $1:"mostvisitedresort"::varchar as mostvisitedresort, $1:"onlysingleresortkey"::varchar as onlysingleresortkey, $1:"partnerresortscannerflag"::varchar as partnerresortscannerflag, $1:"skierabilitylabel"::varchar as skierabilitylabel, $1:"totalseasonsscanned"::varchar as totalseasonsscanned, $1:"visitmostinpeak"::varchar as visitmostinpeak, $1:"age"::varchar as age, $1:"avgvisitperseason"::varchar as avgvisitperseason, $1:"resortsvisited"::varchar as resortsvisited, $1:"subseasonsperyear"::varchar as subseasonsperyear, $1:"totalvisits"::varchar as totalvisits    from @LTBPLocalRunTest/projects/LTBP/FY23/experiments/ltbp_nbs_testing/training_data/None""".replace(' ', '').replace('\n', ''))

In [ ]:
#| export


def read_sfQueries_txt_sql_file(
    file_name: str  # sql file name to read
):
    """Simple utilty to read query files"""
    with open(os.path.join(files.__path__[0], 'sql_files', file_name)) as f:
        read_data = ''.join(f.readlines())
        f.close()
    return read_data

In [ ]:
#| export


def return_sf_type(
    dtype: str,  # data type from a df in string form
    varchar: bool = True  # to default all variables to VARCHAR
):
    """
    simple function to convert dytpes to snowflake dtypes this
    will be come a very useful thing to have as this will dtype
    """
    if varchar is True:
        dtype = 'VARCHAR'
    elif 'int' in dtype.lower():
        dtype = 'NUMBER'
    elif 'float' in dtype.lower():
        dtype = 'FLOAT'
    elif 'object' in dtype.lower():
        dtype = 'VARCHAR'
    elif 'bool' in dtype.lower():
        dtype = 'VARCHAR'  # TODO: Limitation found before change once resloved by sf
    elif 'date' in dtype.lower():
        dtype = 'DATETIME'  # TODO: Might break with certain datetimes most generic
    else:
        logging.error('odd dtype not seen needs to be resloved...')
        sys.exit()
    return dtype

In [ ]:
#| export


def snowflake_query(sfAccount: str = os.environ.get('sfAccount', None),
                    sfUser: str = os.environ.get('sfUser', None),
                    sfPswd: str = os.environ.get('sfPswd', None),
                    sfWarehouse: str = os.environ.get('sfWarehouse', None),
                    sfDatabase: str = os.environ.get('sfDatabase', None),
                    sfSchema: str = os.environ.get('sfSchema', None),
                    sfRole: str = os.environ.get('sfRole', None)):
    """Easy Connection To SnowFlake When Environs are set"""
    sf = Snowflake(sfAccount, sfUser, sfPswd, sfWarehouse,
                   sfDatabase, sfSchema, sfRole)
    return sf

In [ ]:
#| export


def query_feature_sets_to_adls_parquet_sf_fs(
    sf_connection,  # established snowflake connection
    sf_query: str,  # sql query desired to be pushed to adls
    azure_account: str,  # azure acount name
    azure_container: str,  # azure container name to push results to
    data_lake_path: str,  # adls file path location to dump files to
    data_lake_sas_token: str,  # most project use os.environ["DATALAKE_SAS_TOKEN_SECRET"]
    partition_by: str = None,  # save data in a paritions manner
    max_file_size: str = "3200000",  # choose the file size you would like to save your parquet files
    header: str = "True",  # saving with headers or not needs to be a string
    over_write: str = "True",  # if file(s) exisit this will earse exisiting files
):
    # Creating Query to create ADLS Stage for Snowflake
    url = f"azure://{azure_account}.blob.core.windows.net/{azure_container}/{data_lake_path}"
    sf_to_adls_query = sf_to_adls_url_query_generator(
        azure_path=url,
        azure_sas_token=data_lake_sas_token,
        sf_query=sf_query,
        max_file_size=max_file_size,
        file_type="parquet",
        partition_by=partition_by,
        header=header,
        overwrite=over_write,
    )
    # Execute
    _ = sf_connection.run_sql_str(sf_to_adls_query)
    logging.info(f"data has been delivered from sf to adls")

In [ ]:
#| export


def clean_special_chars(text):
    """
    small nlp clean up tool to take odd characters that could be
    in vendor data inside of column names and then replaces empty
    spaces with ``_``

    Args:
    * text (str): dataframe column names as strings

    Returns:
    * str: clean column name
    """
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'  # noqa:
    punct += '©^®` <→°€™› ♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'  # noqa:
    for p in punct:
        text = text.replace(p, '')
        text = text.replace(' ', '')
    return text


In [ ]:
#| export


def select_multi_input_udfs(feature_dict: dict,
                            udf_inputs: dict,
                            query: str,
                            sf_database: str,
                            sf_schema: str,
                            iteration: int,
                            exp_name: str):
    """
    utility function called by `pull_features_from_snowflake`
    to create multi input snowflake udf calls UDF_NAME(Input 1, Input 2)
    a common example is at the ecid grain ISEPICMIXACTIVATED(ECID, 20201001, 20211001)

    Args:
        feature_dict (dict): multi key dictionary of features with
            information the function will end up using to create a dynamic call
        udf_inputs (dict): all information needed for the udf inputs to be created.
        query (str): query string being manipulated
        sf_database (str): snowflake database the udfs live in
        sf_schema (str): snowflake schema the udfs live in
        iteration (int): iteration give the function the ability to know what inputs
            are going to be passed to the udf

    Returns:
        str: manipulated query string
    """
    i = 1
    udf_grain = ', '.join([str(v) for v in udf_inputs['UDF_GRAIN']])
    udf_location = f'{sf_database}.{sf_schema}'
    for temp_feat in feature_dict.keys():
        i += 1
        feat_dict = feature_dict[temp_feat]
        input_def = udf_inputs[feat_dict['input_definition']][exp_name][feat_dict['input_type']][iteration]
        input_def = [input_def] if not isinstance(input_def, list) else input_def
        inputs = ', '.join([str(v) for v in input_def])
        inputs = ', ' + inputs if len(inputs) > 0 else inputs
        result = ''
        if 'iterable_inputs' in feat_dict.keys():
            for iter_idx, iter_input in enumerate(feat_dict['iterable_inputs']):
                if iter_idx + 1 < len(feat_dict['iterable_inputs']):
                    result += f", {udf_location}.{feat_dict['udf_name']}({udf_grain}{inputs}, {iter_input}) as {temp_feat}_{clean_special_chars(iter_input)}\n"
                else:
                    result += f", {udf_location}.{feat_dict['udf_name']}({udf_grain}{inputs}, {iter_input}) as {temp_feat}_{clean_special_chars(iter_input)}\n"
        else:
            if i > len(feature_dict.keys()):
                result = f", {udf_location}.{feat_dict['udf_name']}({udf_grain}{inputs}) as {temp_feat}\n"
            else:
                result = f", {udf_location}.{feat_dict['udf_name']}({udf_grain}{inputs}) as {temp_feat}"
        query = query.replace('<FEATURES>', result + '<FEATURES>')

    query = query.replace('<FEATURES>', '')
    return query

These functions are small twist on MSU already created calls to incorporate experimentation names to the mix to allow for sequential or concurrent calls to python scripts to allow for a smoother and faster iteration of experiments to happen.

In [ ]:
#| export


def pull_features_from_snowflake(
    feature_dict: dict,
    udf_inputs: dict,
    filepath_to_grain_list_query: str = None,
    sf_database: str = 'MACHINELEARNINGFEATURES',
    sf_schema: str = 'PROD',
    feature_table_join: str = None,
    extra_statement: str = None,
    experiment_name: str = 'BASELINE'
):
    """
    a function to allow a user dynamically create snowflake queries that
    creates a feature set that will be transformed into inputs that are
    suitable for ML like task. this function starts with a pre-determined list
    that will define a grain (most commonly used at the ECID grain) that will
    dynamically call snowflake sql udf(s) and possibliy join a table that has
    static like features that will create a dataset.

    To have more understanding of this function and the useability please visit
    (https://vailresorts.gitlab.io/data-science/machine_learning_utilities/)

    Args:
        feature_dict (dict): this dictionary will have information on all the features
            a user is looking to gather in one feature set. Each feature will need a feature_type,
            input_type, input_definition, and a udf_name.
        udf_inputs (dict): this dictionary will have all the information needed for non "static"
            like features that are being created via the joined table. This dictionary will need to have
            a key UDF_GRAIN, FEATURE, LABEL, BASE_QUERY.
        filepath_to_grain_list_query (str, optional): file location of sql files to read.
        sf_database (str, optional): snowflake database UDFs live. Defaults to 'MACHINELEARNINGFEATURES'.
        sf_schema (str, optional): snowflake schema UDFs live. Defaults to 'PROD'.
        feature_table_join (str, optional): a table other than the feature store to join to this dynamic
            call that will have all the static features needed to complete the feature set note the table alias
            on the join statment must be joined. Defaults to None.
        extra_statement (str, optional): allows for additional sql statements i.e LIMIT 1000. Defaults to None.

    Returns:
        str: mainpulated string query ready to be sent to snowflake
    """
    static_features = {k: v for k, v in feature_dict.items() if "STATIC" in v.values()
                       if experiment_name in v['experiment_list']}
    temporal_features = {k: v for k, v in feature_dict.items() if "TEMP" in v.values()
                         if experiment_name in v['experiment_list']}
    logging.info(f'static features in data set: \n {list(static_features.keys())}')
    logging.info(f'temporal features in data set: \n {list(temporal_features.keys())}')
    query_structure = """select
    <MODEL_GRAIN>
    <FEATURES>
    <ADDITIONAL_COLUMNS>
    from
        (base_query) base
    """.replace(" ", "")

    # Bring In all Information From Base Query
    base_query = query_structure.replace("<MODEL_GRAIN>", "base.*")
    if feature_table_join is None:
        join_feat_table = "inner join machinelearningfeatures.prod.featurestore_ecid joined on joined.ecid = base.ecid"
    else:
        join_feat_table = feature_table_join
    base_query = base_query + join_feat_table if len(static_features) > 0 else base_query

    # functional call
    if len(static_features) > 0:
        base_query = select_static_features(feature_dict=static_features,
                                            query=base_query)
        logging.info(f"Finished appending static features")

    final_query = ''
    for i in range(len(udf_inputs['BASE_QUERY'][experiment_name].keys())):
        # functional call
        logging.info(f'reading {udf_inputs["BASE_QUERY"][experiment_name][i]} for base query...')
        feature_query = create_base_query(
            file_path=os.path.join(filepath_to_grain_list_query, udf_inputs["BASE_QUERY"][experiment_name][i]),
            query=base_query)

        # functional call
        if len(temporal_features) > 0:
            feature_query = select_multi_input_udfs(feature_dict=temporal_features,
                                                    udf_inputs=udf_inputs,
                                                    query=feature_query,
                                                    sf_database=sf_database,
                                                    sf_schema=sf_schema,
                                                    iteration=i,
                                                    exp_name=experiment_name)

        # functional call
        if 'ADDITIONAL_COLUMNS' in udf_inputs:
            feature_query = select_additional_features(feature_dict=udf_inputs['ADDITIONAL_COLUMNS'],
                                                       query=feature_query,
                                                       iteration=i)
        else:
            feature_query = feature_query.replace('<ADDITIONAL_COLUMNS>', '')

        if i + 1 < len(udf_inputs['BASE_QUERY'][experiment_name].keys()):
            feature_query += " \nUNION ALL\n"
        final_query += feature_query

    if extra_statement is not None:
        final_query += f'\n{extra_statement}'

    logging.info(f'final query output: \n {final_query}')
    return final_query

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()